<a href="https://colab.research.google.com/github/kimhyeongmin-khu/MLMovieProj/blob/main/week3%EC%98%88%EC%B8%A1_nonlinear_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#RandomForestRegressor 최적의 하이퍼파라미터 탐색
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import RFE

# 난수 생성기의 시드 고정
np.random.seed(42)

# CSV 파일 읽기
file_path = '0530_dataset/preprosessing_ver15.csv'  # 파일 경로를 지정하세요
df = pd.read_csv(file_path)

# 타겟 변수 설정
target_variable = 'w3_ac_au'

# 기본 피처 세트 정의 (사용자가 제공)
user_selected_features = [
   'w2_ac_au', 'w2_rank', 'w2_av_sc', 'w2_av_sales', 'w2_df_rank',
   'di_ca_au_y3', 'Distributors_mv_au_y3', 'actor_mv_au_y3', 'Week2_Avg',
   'Over_12', 'Over_15', 'General_Audience', 'No_Youth', 'Is_High_Season',
   'Others', 'USA', 'Korea'
]

# 제공된 기본 피처 세트를 사용
X = df[user_selected_features].copy()
y = df[target_variable]

# RFE를 사용한 피처 선택
model = RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=2, min_samples_leaf=1, random_state=42)
rfe = RFE(estimator=model, n_features_to_select=10)
fit = rfe.fit(X, y)

# 선택된 피처들
selected_features = np.array(user_selected_features)[fit.support_]
print(f"Selected features: {selected_features}")

# 선택된 피처들만 사용하여 데이터프레임 구성
X_selected = X.loc[:, fit.support_]

# 교차 검증을 통한 모델 평가 및 상위 오차 데이터 추출
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)
mae_scores = []
mse_scores = []
rmse_scores = []
mape_scores = []
r_squared_scores = []

# 오차 비율이 큰 상위 데이터를 저장할 리스트
top_errors = []

for train_idx, val_idx in k_fold.split(X_selected):
    X_train_fold, X_val_fold = X_selected.iloc[train_idx], X_selected.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    model.fit(X_train_fold, y_train_fold)
    y_val_pred = model.predict(X_val_fold)

    mae_scores.append(mean_absolute_error(y_val_fold, y_val_pred))
    mse_scores.append(mean_squared_error(y_val_fold, y_val_pred))
    rmse_scores.append(np.sqrt(mean_squared_error(y_val_fold, y_val_pred)))
    mape_scores.append(np.mean(np.abs((y_val_fold - y_val_pred) / y_val_fold)) * 100)
    r_squared_scores.append(r2_score(y_val_fold, y_val_pred))

    # 오차 비율 계산 및 상위 오차 데이터 저장
    error_ratios = np.abs((y_val_fold - y_val_pred) / y_val_fold)
    top_error_idx = np.argmax(error_ratios)
    top_errors.append({
        'Actual': y_val_fold.iloc[top_error_idx],
        'Predicted': y_val_pred[top_error_idx],
        'Error Ratio': error_ratios.iloc[top_error_idx]
    })

# 교차 검증 평가 결과 출력
results = {
    'MAE': np.mean(mae_scores),
    'MSE': np.mean(mse_scores),
    'RMSE': np.mean(rmse_scores),
    'MAPE': np.mean(mape_scores),
    'R2': np.mean(r_squared_scores)
}
print(results)

# 피처 중요도 출력
importances = model.feature_importances_
for feature, importance in zip(selected_features, importances):
    print(f"{feature}: {importance:.4f}")

# 상위 오차 비율 데이터 출력
top_errors_sorted = sorted(top_errors, key=lambda x: x['Error Ratio'], reverse=True)[:10]
print("Top 10 Error Ratios:")
for error_data in top_errors_sorted:
    print(f"Actual: {error_data['Actual']}, Predicted: {error_data['Predicted']}, Error Ratio: {error_data['Error Ratio']}")


In [ ]:
#KNeighborsRegressor 최적의 하이퍼파라미터 탐색
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_absolute_percentage_error

# 난수 생성기의 시드 고정
np.random.seed(42)

# CSV 파일 읽기
file_path = '0530_dataset/preprosessing_ver15.csv'  # 파일 경로를 지정하세요
df = pd.read_csv(file_path)

# 타겟 변수 설정
target_variable = 'w3_ac_au'

# 숫자형 데이터만 선택
numeric_cols = df.select_dtypes(include=[np.number]).columns

# 제외할 열들
exclude_cols = ['w1_slope', 'w1_df_rank', 'w1_mean_diff', 'w2_slope', 'w2_df_rank', 'w2_mean_diff']

# 범주형 열들 (예를 들어 사용자가 지정한 범주형 열들)
categorical_cols = ['Is_High_Season', 'Over_12', 'Over_15', 'General_Audience', 'No_Youth', 'USA', 'Korea', 'Others']

# 제외할 열들을 뺀 나머지 열들
cols_to_transform = [col for col in numeric_cols if col not in exclude_cols]

# 나머지 열들에 대해 0 이하의 값을 작은 양수로 대체
df[cols_to_transform] = df[cols_to_transform].apply(lambda x: np.where(x > 0, x, 1e-6))

# 기본 피처 세트 정의 (사용자가 제공)
user_selected_features = [
   'w2_ac_au', 'w2_rank', 'w2_av_sc', 'w2_av_sales',  'w2_df_rank',
   'di_ca_au_y3', 'Distributors_mv_au_y3', 'actor_mv_au_y3', 'Week2_Avg',
   'Over_12', 'Over_15', 'General_Audience', 'No_Youth', 'Is_High_Season',
   'Others', 'USA', 'Korea'
]

# 제공된 기본 피처 세트를 사용
X_transformed = df[user_selected_features].copy()

# 로그 변환 (exclude_cols 제외)
numeric_features_to_log = [col for col in user_selected_features if col not in categorical_cols + exclude_cols]
X_transformed[numeric_features_to_log] = np.log1p(X_transformed[numeric_features_to_log])
y_transformed = np.log1p(df[target_variable])

# 수치형 열만 스케일링
numeric_features = [col for col in user_selected_features if col not in categorical_cols]
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_transformed[numeric_features])

# 스케일링 된 수치형 데이터와 범주형 데이터를 다시 결합
X_combined = np.concatenate([X_numeric_scaled, X_transformed[categorical_cols].values], axis=1)

# 전체 피처 이름 리스트
combined_feature_names = numeric_features + categorical_cols

# KNN 회귀 모델 정의
knn = KNeighborsRegressor()

# 그리드 서치로 최적의 하이퍼파라미터 탐색
param_grid = {'n_neighbors': np.arange(1, 31)}
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
grid_search = GridSearchCV(knn, param_grid, cv=KFold(n_splits=10, shuffle=True, random_state=42), scoring=scorer)

# 그리드 서치 수행
grid_search.fit(X_combined, y_transformed)

# 최적의 하이퍼파라미터 출력
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best MAPE: {-grid_search.best_score_}")

# 최적의 모델로 예측 수행
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_combined)
y_pred_original = np.expm1(y_pred)
y_true_original = np.expm1(y_transformed)

# 전체 데이터에서 MAPE 계산
overall_mape = mean_absolute_percentage_error(y_true_original, y_pred_original)
print(f"Overall MAPE: {overall_mape}")

# 피처 중요도는 KNN에서는 제공되지 않으므로 생략

# 최적의 하이퍼파라미터 찾기
n_neighbors = grid_search.best_params_['n_neighbors']
print(f"Optimal number of neighbors: {n_neighbors}")

# 상위 오차 비율 데이터 출력
error_ratios = np.abs((y_true_original - y_pred_original) / y_true_original)
top_errors_idx = np.argsort(error_ratios)[-10:]
top_errors = pd.DataFrame({
    'Actual': y_true_original[top_errors_idx],
    'Predicted': y_pred_original[top_errors_idx],
    'Error Ratio': error_ratios[top_errors_idx]
})

print("Top 10 Error Ratios:")
print(top_errors)


In [ ]:
#xgb 최적의 하이퍼파라미터 탐색
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
import xgboost as xgb

# 난수 생성기의 시드 고정
np.random.seed(42)

# CSV 파일 읽기
file_path = '0530_dataset\preprosessing_ver15.csv'  # 파일 경로를 지정하세요
df = pd.read_csv(file_path)

# 타겟 변수 설정
target_variable = 'w3_ac_au'

# 숫자형 데이터만 선택
numeric_cols = df.select_dtypes(include=[np.number]).columns

# 제외할 열들
exclude_cols = ['w1_slope', 'w1_df_rank', 'w1_mean_diff', 'w2_slope', 'w2_df_rank', 'w2_mean_diff']

# 범주형 열들 (예를 들어 사용자가 지정한 범주형 열들)
categorical_cols = ['Is_High_Season', 'Over_12', 'Over_15', 'General_Audience', 'No_Youth', 'USA', 'Korea', 'Others']

# 제외할 열들을 뺀 나머지 열들
cols_to_transform = [col for col in numeric_cols if col not in exclude_cols]

# 나머지 열들에 대해 0 이하의 값을 작은 양수로 대체
df[cols_to_transform] = df[cols_to_transform].apply(lambda x: np.where(x > 0, x, 1e-6))

# 기본 피처 세트 정의 (사용자가 제공)
user_selected_features = [
    'w2_ac_au', 'w2_rank', 'w2_av_sc', 'w2_av_sales',  'w2_df_rank',
    'di_ca_au_y3', 'Distributors_mv_au_y3', 'actor_mv_au_y3', 'Week2_Avg',
    'Over_12', 'Over_15', 'General_Audience', 'No_Youth', 'Is_High_Season',
    'Others', 'USA', 'Korea'
]

# 제공된 기본 피처 세트를 사용
X_transformed = df[user_selected_features].copy()

# 로그 변환 (exclude_cols 제외)
numeric_features_to_log = [col for col in user_selected_features if col not in categorical_cols + exclude_cols]
X_transformed[numeric_features_to_log] = np.log1p(X_transformed[numeric_features_to_log])
y_transformed = np.log1p(df[target_variable])

# 수치형 열만 스케일링
numeric_features = [col for col in user_selected_features if col not in categorical_cols]
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_transformed[numeric_features])

# 스케일링 된 수치형 데이터와 범주형 데이터를 다시 결합
X_combined = np.concatenate([X_numeric_scaled, X_transformed[categorical_cols].values], axis=1)

# 전체 피처 이름 리스트
combined_feature_names = numeric_features + categorical_cols

# XGBoost 회귀 모델 정의
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# 그리드 서치로 최적의 하이퍼파라미터 탐색
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
grid_search = GridSearchCV(xgb_model, param_grid, cv=KFold(n_splits=10, shuffle=True, random_state=42), scoring=scorer)

# 그리드 서치 수행
grid_search.fit(X_combined, y_transformed)

# 최적의 하이퍼파라미터 출력
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best MAPE: {-grid_search.best_score_}")


In [ ]:
#RandomForestRegressor예측
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import RFE

# 난수 생성기의 시드 고정
np.random.seed(42)

# CSV 파일 읽기
file_path = '0530_dataset/preprosessing_ver15.csv'  # 파일 경로를 지정하세요
df = pd.read_csv(file_path)

# 타겟 변수 설정
target_variable = 'w3_ac_au'

# 숫자형 데이터만 선택
numeric_cols = df.select_dtypes(include=[np.number]).columns

# 제외할 열들
exclude_cols = ['w1_slope', 'w1_df_rank', 'w1_mean_diff', 'w2_slope', 'w2_df_rank', 'w2_mean_diff']

# 범주형 열들 (예를 들어 사용자가 지정한 범주형 열들)
categorical_cols = ['Is_High_Season', 'Over_12', 'Over_15', 'General_Audience', 'No_Youth', 'USA', 'Korea', 'Others']

# 제외할 열들을 뺀 나머지 열들
cols_to_transform = [col for col in numeric_cols if col not in exclude_cols]

# 나머지 열들에 대해 0 이하의 값을 작은 양수로 대체
df[cols_to_transform] = df[cols_to_transform].apply(lambda x: np.where(x > 0, x, 1e-6))

# 기본 피처 세트 정의 (사용자가 제공)
user_selected_features = [
   'w2_ac_au', 'w2_rank', 'w2_av_sc', 'w2_av_sales', 'w2_df_rank',
   'di_ca_au_y3', 'Distributors_mv_au_y3', 'actor_mv_au_y3', 'Week2_Avg',
   'Over_12', 'Over_15', 'General_Audience', 'No_Youth', 'Is_High_Season',
   'Others', 'USA', 'Korea'
]

# 제공된 기본 피처 세트를 사용
X_transformed = df[user_selected_features].copy()

# 로그 변환 (exclude_cols 제외)
numeric_features_to_log = [col for col in user_selected_features if col not in categorical_cols + exclude_cols]
X_transformed[numeric_features_to_log] = np.log1p(X_transformed[numeric_features_to_log])
y_transformed = np.log1p(df[target_variable])

# 수치형 열만 스케일링
numeric_features = [col for col in user_selected_features if col not in categorical_cols]
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_transformed[numeric_features])

# 스케일링 된 수치형 데이터와 범주형 데이터를 다시 결합
X_combined = np.concatenate([X_numeric_scaled, X_transformed[categorical_cols].values], axis=1)

# 전체 피처 이름 리스트
combined_feature_names = numeric_features + categorical_cols

# RFE를 사용한 피처 선택
model = RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=2, min_samples_leaf=1, random_state=42)
rfe = RFE(estimator=model, n_features_to_select=10)
fit = rfe.fit(X_combined, y_transformed)

# 선택된 피처들
selected_features = np.array(combined_feature_names)[fit.support_]
print(f"Selected features: {selected_features}")

# 선택된 피처들만 사용하여 데이터프레임 구성
X_selected = X_combined[:, fit.support_]

# 교차 검증을 통한 모델 평가 및 상위 오차 데이터 추출
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)
mae_scores = []
mse_scores = []
rmse_scores = []
mape_scores = []
r_squared_scores = []

# 오차 비율이 큰 상위 데이터를 저장할 리스트
top_errors = []

for train_idx, val_idx in k_fold.split(X_selected):
    X_train_fold, X_val_fold = X_selected[train_idx], X_selected[val_idx]
    y_train_fold, y_val_fold = y_transformed.iloc[train_idx], y_transformed.iloc[val_idx]

    model.fit(X_train_fold, y_train_fold)
    y_val_pred = model.predict(X_val_fold)

    # 예측값과 실제값을 원래 스케일로 되돌리기
    y_val_pred_exp = np.expm1(y_val_pred)
    y_val_exp = np.expm1(y_val_fold)

    mae_scores.append(mean_absolute_error(y_val_exp, y_val_pred_exp))
    mse_scores.append(mean_squared_error(y_val_exp, y_val_pred_exp))
    rmse_scores.append(np.sqrt(mean_squared_error(y_val_exp, y_val_pred_exp)))
    mape_scores.append(np.mean(np.abs((y_val_exp - y_val_pred_exp) / y_val_exp)) * 100)
    r_squared_scores.append(r2_score(y_val_exp, y_val_pred_exp))

    # 오차 비율 계산 및 상위 오차 데이터 저장
    error_ratios = np.abs((y_val_exp - y_val_pred_exp) / y_val_exp)
    top_error_idx = np.argmax(error_ratios)
    top_errors.append({
        'Actual': y_val_exp.iloc[top_error_idx],
        'Predicted': y_val_pred_exp[top_error_idx],
        'Error Ratio': error_ratios.iloc[top_error_idx]
    })

# 교차 검증 평가 결과 출력
results = {
    'MAE': np.mean(mae_scores),
    'MSE': np.mean(mse_scores),
    'RMSE': np.mean(rmse_scores),
    'MAPE': np.mean(mape_scores),
    'R2': np.mean(r_squared_scores)
}
print(results)

# 피처 중요도 출력
importances = model.feature_importances_
for feature, importance in zip(selected_features, importances):
    print(f"{feature}: {importance:.4f}")

# 상위 오차 비율 데이터 출력
top_errors_sorted = sorted(top_errors, key=lambda x: x['Error Ratio'], reverse=True)[:10]
print("Top 10 Error Ratios:")
for error_data in top_errors_sorted:
    print(f"Actual: {error_data['Actual']}, Predicted: {error_data['Predicted']}, Error Ratio: {error_data['Error Ratio']}")


In [ ]:
#KNeighborsRegressor예측
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsRegressor

# KNN 모델 사용
knn_model = KNeighborsRegressor(n_neighbors=8)

# SequentialFeatureSelector 사용
sfs = SequentialFeatureSelector(knn_model, n_features_to_select=10, direction='forward')
sfs.fit(X_combined, y_transformed)

# 선택된 피처들
selected_features_sfs = np.array(combined_feature_names)[sfs.get_support()]
print(f"Selected features with SequentialFeatureSelector: {selected_features_sfs}")

# 선택된 피처들만 사용하여 데이터프레임 구성
X_selected_sfs = X_combined[:, sfs.get_support()]

# 교차 검증을 통한 KNN 모델 평가
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)
mae_scores_sfs = []
mse_scores_sfs = []
rmse_scores_sfs = []
mape_scores_sfs = []
r_squared_scores_sfs = []

# 오차 비율이 큰 상위 데이터를 저장할 리스트
top_errors = []

for train_idx, val_idx in k_fold.split(X_selected_sfs):
    X_train_fold, X_val_fold = X_selected_sfs[train_idx], X_selected_sfs[val_idx]
    y_train_fold, y_val_fold = y_transformed.iloc[train_idx], y_transformed.iloc[val_idx]

    knn_model.fit(X_train_fold, y_train_fold)
    y_val_pred = knn_model.predict(X_val_fold)

    # 예측값과 실제값을 원래 스케일로 되돌리기
    y_val_pred_exp = np.expm1(y_val_pred)
    y_val_exp = np.expm1(y_val_fold)

    mae_scores_sfs.append(mean_absolute_error(y_val_exp, y_val_pred_exp))
    mse_scores_sfs.append(mean_squared_error(y_val_exp, y_val_pred_exp))
    rmse_scores_sfs.append(np.sqrt(mean_squared_error(y_val_exp, y_val_pred_exp)))
    mape_scores_sfs.append(np.mean(np.abs((y_val_exp - y_val_pred_exp) / y_val_exp)) * 100)
    r_squared_scores_sfs.append(r2_score(y_val_exp, y_val_pred_exp))

    # 오차 비율 계산 및 상위 오차 데이터 저장
    error_ratios = np.abs((y_val_exp - y_val_pred_exp) / y_val_exp)
    top_error_idx = np.argmax(error_ratios)
    top_errors.append({
        'Actual': y_val_exp.iloc[top_error_idx],
        'Predicted': y_val_pred_exp[top_error_idx],
        'Error Ratio': error_ratios.iloc[top_error_idx]
    })

# 교차 검증 평가 결과 출력
results_sfs = {
    'MAE': np.mean(mae_scores_sfs),
    'MSE': np.mean(mse_scores_sfs),
    'RMSE': np.mean(rmse_scores_sfs),
    'MAPE': np.mean(mape_scores_sfs),
    'R2': np.mean(r_squared_scores_sfs)
}
print(f"KNN Results with SequentialFeatureSelector: {results_sfs}")

# 상위 오차 비율 데이터 출력
top_errors_sorted = sorted(top_errors, key=lambda x: x['Error Ratio'], reverse=True)[:10]
print("Top 10 Error Ratios:")
for error_data in top_errors_sorted:
    print(f"Actual: {error_data['Actual']}, Predicted: {error_data['Predicted']}, Error Ratio: {error_data['Error Ratio']}")

In [ ]:
#XGBRegressor예측
import xgboost as xgb
from sklearn.feature_selection import RFE

# XGBoost 모델 사용
xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)
rfe_xgb = RFE(estimator=xgb_model, n_features_to_select=10)
fit_xgb = rfe_xgb.fit(X_combined, y_transformed)

# 선택된 피처들
selected_features_xgb = np.array(combined_feature_names)[fit_xgb.support_]
print(f"Selected features with XGBoost: {selected_features_xgb}")

# 선택된 피처들만 사용하여 데이터프레임 구성
X_selected_xgb = X_combined[:, fit_xgb.support_]

# 교차 검증을 통한 XGBoost 모델 평가
mae_scores_xgb = []
mse_scores_xgb = []
rmse_scores_xgb = []
mape_scores_xgb = []
r_squared_scores_xgb = []

# 오차 비율이 큰 상위 데이터를 저장할 리스트
top_errors_g = []

for train_idx, val_idx in k_fold.split(X_selected_xgb):
    X_train_fold, X_val_fold = X_selected_xgb[train_idx], X_selected_xgb[val_idx]
    y_train_fold, y_val_fold = y_transformed.iloc[train_idx], y_transformed.iloc[val_idx]

    xgb_model.fit(X_train_fold, y_train_fold)
    y_val_pred = xgb_model.predict(X_val_fold)

    # 예측값과 실제값을 원래 스케일로 되돌리기
    y_val_pred_exp = np.expm1(y_val_pred)
    y_val_exp = np.expm1(y_val_fold)

    mae_scores_xgb.append(mean_absolute_error(y_val_exp, y_val_pred_exp))
    mse_scores_xgb.append(mean_squared_error(y_val_exp, y_val_pred_exp))
    rmse_scores_xgb.append(np.sqrt(mean_squared_error(y_val_exp, y_val_pred_exp)))
    mape_scores_xgb.append(np.mean(np.abs((y_val_exp - y_val_pred_exp) / y_val_exp)) * 100)
    r_squared_scores_xgb.append(r2_score(y_val_exp, y_val_pred_exp))

    # 오차 비율 계산 및 상위 오차 데이터 저장
    error_ratios_g = np.abs((y_val_exp - y_val_pred_exp) / y_val_exp)
    top_error_idx_g = np.argmax(error_ratios_g)
    top_errors_g.append({
        'Actual': y_val_exp.iloc[top_error_idx_g],
        'Predicted': y_val_pred_exp[top_error_idx_g],
        'Error Ratio': error_ratios_g.iloc[top_error_idx_g]
    })

# 교차 검증 평가 결과 출력
results_xgb = {
    'MAE': np.mean(mae_scores_xgb),
    'MSE': np.mean(mse_scores_xgb),
    'RMSE': np.mean(rmse_scores_xgb),
    'MAPE': np.mean(mape_scores_xgb),
    'R2': np.mean(r_squared_scores_xgb)
}
print(f"XGBoost Results: {results_xgb}")
# 상위 오차 비율 데이터 출력
top_errors_sorted_g = sorted(top_errors_g, key=lambda x: x['Error Ratio'], reverse=True)[:10]
print("Top 10 Error Ratios:")
for error_data in top_errors_sorted_g:
    print(f"Actual: {error_data['Actual']}, Predicted: {error_data['Predicted']}, Error Ratio: {error_data['Error Ratio']}")